In [1]:
# !pip install datasets transformers

In [2]:
import torch
device = 'cuda:pick_a_device' if torch.cuda.is_available() else 'cpu'

In [3]:
from collections import defaultdict
import os
import datetime
from pathlib import Path
import json
from tqdm import tqdm

from torch import nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler

# import datasets
# from datasets import load_dataset
from transformers import AutoTokenizer

import utils.load_corpus as load_corpus
import utils.losses as l

import utils.RNN as RNN
import model.pick_a_model as rotator

import importlib

In [4]:
PATH = "data"
folder_path = Path(f"limanet")
os.makedirs(PATH/folder_path, exist_ok=True)

In [5]:
tokenizer_type = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)
text = ["I love bird.", "Hi, I'm Bob."]
tokenized = tokenizer(text, return_tensors='pt', padding=True)
tokenized['input_ids']

tensor([[ 101, 1045, 2293, 4743, 1012,  102,    0,    0,    0],
        [ 101, 7632, 1010, 1045, 1005, 1049, 3960, 1012,  102]])

In [6]:
# def transform(batch):
#   tokenized = tokenizer(batch['text'], return_tensors='pt', padding=True, add_special_tokens=False)
#   packed = torch.nn.utils.rnn.pack_padded_sequence(tokenized['input_ids'], tokenized['attention_mask'].sum(dim=1), enforce_sorted=False, batch_first=True)
#   to_return = {
#     'data': packed.data,
#     'batch_sizes': packed.batch_sizes,
#     'attention_mask': tokenized['attention_mask'],
#     'sorted_indices': packed.sorted_indices,
#     'unsorted_indices': packed.unsorted_indices
#   }
#   return to_return

# dataset.set_transform(transform)

# dataset['train'][:2]

In [7]:
# # test
# rotator_depth = 3

# import importlib
# rotator = importlib.reload(rotator)
# RNN = importlib.reload(RNN)

# model = RNN.RNN(rotator.Rotator(tokenizer.vocab_size, depth=rotator_depth, dim=128, num_heads=32, hippo_dim=16, num_hippo_heads=8))#.to(device)
# model.eval()
# batch = dataset['train'][:3]

# gen = model(**batch)
# predicts, targets = [torch.concat(res) for res in zip(*gen)]
# predicts.shape, targets.shape


In [8]:
%%time

batch_size = 128
shuffle = True

import importlib
load_corpus = importlib.reload(load_corpus)

# dataset = load_corpus.load_bookcorpus(tokenizer, batch_size=batch_size, shuffle=shuffle, cache_dir='~/data1-0756727/cache/huggingface')
# dataset = load_corpus.load_msmarco(tokenizer, batch_size=batch_size, shuffle=shuffle, cache_dir='~/data1-0756727/cache/huggingface')
dataset = load_corpus.Load_wiki(tokenizer, batch_size=batch_size, shuffle=shuffle, cache_dir='~/data1-0756727/cache/huggingface')
print(dataset.dataset['train'][:2])
train_loader = dataset.loader()

n = next(iter(train_loader))
# print(tokenizer.decode(n['data']))
{key: n[key].shape for key in n}

/home/adam/miniconda3/envs/lima/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


{'data': tensor([9617, 1010, 1999,  ..., 2451, 1516, 2500]), 'batch_sizes': tensor([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'sorted_indices': tensor([0, 1, 2, 3, 4, 5, 6, 7]), 'unsorted_indices': tensor([0, 1, 2, 3, 4, 5, 6, 7])}
CPU times: user 4.03 s, sys: 4.55 s, total: 8.58 s
Wall time: 5.57 s


{'data': torch.Size([38251]),
 'batch_sizes': torch.Size([128]),
 'attention_mask': torch.Size([128, 512]),
 'sorted_indices': torch.Size([343]),
 'unsorted_indices': torch.Size([343])}

In [9]:
subpath = '20240624.12:08:14' # 4 ssm_ce_with_bias

batch_num = 25000

learning_rate = 0.01 # for cross entropy
learning_rate = 0.001 # for fine tuning

model_params = {
  'rotator_depth': 3,
  'num_heads': 32,
  'dim': 128,
  'hidden_dim': 32,
  'rotary_denom': 0.5,
  # 'embedding_initialize_method': 'rand'
}
loss_params = {
  'sampling_word_size': 10,
  'margin': 8, # for euclidean
  # 'margin': .3, # for cosine
}

rotator = importlib.reload(rotator)
l = importlib.reload(l)
RNN = importlib.reload(RNN)

RNN_base = RNN.RNN
RNN_base = RNN.RNN_with_bias

# model = RNN_base(rotator.Rotator(tokenizer.vocab_size, **model_params), weight_tying=False).to(device)
# using_loss = l.Complex_mse()
# using_loss = l.Complex_triplet_loss(model)
# using_loss = l.Complex_mse_triplet_loss(model)
# using_loss = l.Complex_mse_squared_triplet_loss(model)
# using_loss = l.Mse_multiplet_loss(model.model, **loss_params)
# using_loss = l.Cos_multiplet_loss(model.model, **loss_params)

model = RNN_base(rotator.Rotator(tokenizer.vocab_size, **model_params), weight_tying=False, return_label_embeddings=False).to(device)
using_loss = l.Sampled_softmax_cross_entropy(model)

# model = RNN_base(rotator.Rotator(tokenizer.vocab_size, **model_params)).to(device)
# using_loss = nn.CrossEntropyLoss()
# using_loss = l.Cross_entropy_l2_embedding(model.model, min_squared_norm=model.model.dim)
# using_loss = l.Cross_entropy_l2_embedding(model.model, l2_squared_weight=1e-3)

model = torch.load(PATH/folder_path/subpath/f'batch_{batch_num}-model.pt', map_location=device)

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, total_iters=10, start_factor=0.01)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.001
    lr: 1e-05
    maximize: False
    weight_decay: 0.01
)

In [10]:
# # testing model output
# batch = next(iter(train_loader))
# batch = {key: batch[key].to(device) for key in batch}

# gen = model(**batch)
# predicts, targets = [torch.concat(res) for res in zip(*gen)]
# losses = using_loss(predicts, targets)
# loss = sum(losses.values())
# loss_dict = {
#   'total': f"{loss.item(): .6f}",
#   **{key: f"{losses[key].item(): .6f}" for key in losses},
# }

# print(loss_dict)
# # del batch, loss
# # torch.cuda.empty_cache()

In [11]:
# def backward_hook(module, gin, gout):
#   print(f"{len(gin)=}, {len(gout)=}")
#   print(*[f"{i=}, {gi.shape=}, {gi.mean()=}, {gi.min()=}, {gi.max()=}" for i, gi in enumerate(gin)], sep='\n')
#   print(*[f"{i=}, {go.shape=}, {go.mean()=}, {go.min()=}, {go.max()=}" for i, go in enumerate(gout)], sep='\n')

# model.limas[0].register_backward_hook(backward_hook)

In [12]:
time_string = datetime.datetime.now(tz=datetime.timezone(datetime.timedelta(hours=8))).strftime('%Y%m%d.%H:%M:%S')
subfolder_path = Path(time_string)

os.makedirs(PATH/folder_path/subfolder_path, exist_ok=True)

with open(PATH/folder_path/subfolder_path/f'parameters.json', 'w') as f:
  f.write(json.dumps({
    'tokenizer_type': tokenizer_type,
    'model': str(model),
    'model_params': model_params,
    'learning_rate': learning_rate,
    'batch_size': batch_size,
    'shuffle': shuffle,
    'loss_type': str(using_loss),
    'loss_params': loss_params,
  }))  
time_string

'20240626.09:08:18'

In [ ]:
num_epochs = 1
save_every_n_batches = 5000

multiloss = 1

# os.makedirs(PATH/folder_path/subfolder_path, exist_ok=True)

loss_history = []

for epoch_num, epoch in enumerate(range(num_epochs)):
  bar = tqdm(train_loader)

  for batch_num, batch in enumerate(bar):

    optimizer.zero_grad()
    
    batch = {key: batch[key].to(device) for key in batch}
    gen = model(**batch)

    if multiloss:
      total_losses = defaultdict(lambda: torch.tensor([0], dtype=torch.float).to(device))
    loss = 0
    while True: 
      results = []
      try:
        for _ in range(100):
          results.append(next(gen))
        predicts, targets = [torch.concat(pred_or_targ) for pred_or_targ in zip(*results)]
        
        if multiloss:
          losses = using_loss(predicts, targets)
          for loss_name, value in losses.items():
            total_losses[loss_name] += value
        else:
          loss += using_loss(predicts, targets)
          
      except StopIteration:
        if len(results) > 0:
          predicts, targets = [torch.concat(pred_or_targ) for pred_or_targ in zip(*results)]
          
          if multiloss:
            losses = using_loss(predicts, targets)
            for loss_name, value in losses.items():
              total_losses[loss_name] += value
          else:
            loss += using_loss(predicts, targets)
        break

    if multiloss:
      loss = sum(total_losses.values())
    loss_dict = {
      'loss': f"{loss.item(): .6f}",
    }
    if multiloss:
      loss_dict |= {key: f"{losses[key].item(): .6f}" for key in losses}
      
    bar.set_postfix(loss_dict)

    # with torch.autograd.detect_anomaly(True):
      # loss.backward()
      # torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=10, norm_type=2)
    loss.backward()
    # break 
    optimizer.step()

    if (batch_num % 100 == 0):
      # torch.cuda.empty_cache()
      loss_history.append([epoch_num, batch_num, loss_dict])
      # print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, batch {batch_num}")

    if (batch_num % 1000 == 999):
      scheduler.step()

    if (batch_num % save_every_n_batches == 0):
      prefix = f"batch_{batch_num}-"
      torch.save(model, PATH/folder_path/subfolder_path/(prefix + f'model.pt'))
        
      with open(PATH/folder_path/subfolder_path/'history.json', 'w') as f:
        f.write(json.dumps(loss_history))  

      # torch.cuda.empty_cache()

    if (batch_num in [100, 500, 1000, 1500, 2000, 3000, 7500, 12500, 17500, 22500]):
        prefix = f"batch_{batch_num}-"
        torch.save(model, PATH/folder_path/subfolder_path/(prefix + 'model.pt'))
        
        with open(PATH/folder_path/subfolder_path/'history.json', 'w') as f:
          f.write(json.dumps(loss_history))  

    # del batch, loss

  5%|▍         | 2315/50459 [33:41<11:31:44,  1.16it/s, loss=10.851339]

In [ ]:
inputs = tokenizer('I love you.', return_tensors='pt', add_special_tokens=False)
histories = [his.expand(4, *[-1]*(len(his.shape)-1)) for his in model.model.initial_states()['h']]
output, next_histories = model.model.forward(inputs['input_ids'][0].to(device), histories)

In [ ]:
output